In [1]:

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
import sys
sys.path.append('./lib')
from lib.cleaning import *
from lib.data_prepration import *
from lib.paralellism import *
from lib.mydoc2vec import *
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import nltk

from gensim.models import Doc2Vec
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
#!pip install scipy==1.12
SEED = 448

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     c:\Users\alire\Downloads\neuefische\ds-
[nltk_data]     capstone\.venv\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
"""
!pip install pandarallel
!pip install nltk 
!python.exe -m pip install --upgrade pip
!pip install gensim
!pip install scipy==1.12 --upgrade"""

In [2]:

keep_cols=['title', 'price', 'helpfulness', 'score', 'time',
       'summary', 'text', 'description', 'authors', 'publisher',
       'publisheddate', 'categories', 'ratingscount']
file_path = '../data/book_review.csv'
data_prep = DataPreparation(file_path)
data_prep.clean_and_preprocess(True)
data_prep.Normalize()
data_prep.data=data_prep.data[keep_cols]

#data_prep.data=data_prep.data.sample(frac=0.3,random_state=SEED)
data_prep.data.categories.value_counts().sum()

2359775

In [5]:
data_prep.data.categories=data_prep.data.categories.str.lower().replace(']','').replace('[','')
data_prep.data.categories.value_counts()

,count
categories,
[Fiction],815445
[Juvenile Fiction],205925
[],115970
[Biography & Autobiography],104284
[Religion],94700
...,...
[Bayous],1
[Bethlehem (Pa.)],1
[Computer-aided design],1


In [12]:
d_not_fic=data_prep.data[data_prep.data.categories!="[Fiction]"].sample(frac=0.3)
d_fic=data_prep.data[data_prep.data.categories=="[Fiction]"].sample(frac=0.12)
d_not_fic.categories.value_counts()

categories
[Fiction]                             244248
[Juvenile Fiction]                     61813
[]                                     34722
[Biography & Autobiography]            31277
[Religion]                             28347
                                       ...  
[Devices (Heraldry)]                       1
[Adult child sexual abuse victims]         1
[Learning disabled youth]                  1
[Water quality]                            1
[Fractures]                                1
Name: count, Length: 3865, dtype: int64

In [ ]:
"""

keep_cols=['title', 'price', 'helpfulness', 'score', 'time',
       'summary', 'text', 'description', 'authors', 'publisher',
       'publisheddate', 'categories', 'ratingscount']
file_path = '../data/one_half.csv'
data_prep = DataPreparation(file_path)
data_prep.read_large_csv()
random.seed(SEED)

data_prep.data=data_prep.data[keep_cols]
data_prep.data"""

In [ ]:
data_prep.data.columns

In [ ]:
len(data_prep.data)

In [ ]:
print(len(data_prep.data))
print(data_prep.data.info())
print(data_prep.data.describe())

In [ ]:
data_prep.data.isnull().sum()

In [ ]:
data_prep.data.score.value_counts()

In [ ]:
data_prep.generate_plots()

In [ ]:
corr_matrix = data_prep.data.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
join_func = lambda x: ', '.join(map(str, x))
data_prep.data['all']=data_prep.data['title'] + data_prep.data['description'] + data_prep.data['authors'].apply(join_func)+data_prep.data['categories'].apply(join_func)
data_prep.data['all']


In [ ]:

_doc2vec = Doc2VecRecommender()
testdata=_doc2vec.train(data_prep.data)


In [ ]:
_doc2vec.model.save("./models/doc2vec_model_hole")

In [ ]:
_doc2vec_load = Doc2VecRecommender(data_prep.data)
_doc2vec_load.load_model('./models/doc2vec_model_hole')


In [ ]:
search='after ice age a lion king is back'
_doc2vec_load.recommend_by_text(search) 

In [ ]:
data_prep.data=data_prep.data.reset_index()
data_prep.data

In [ ]:
data_prep.data.to_csv('../data/cleared_columns.csv')

In [ ]:
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('wordnet')
def preprocess(sentence,lemmatizer,stop_words_temp=stop_words):
    
    tokens = nltk.word_tokenize(sentence,language='english')
    tokens=[word for word in tokens if word not in stop_words_temp]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

lemmatizer = WordNetLemmatizer()

documents = [TaggedDocument(preprocess(doc,lemmatizer), [i]) for i, doc in enumerate(data_prep.data['all'])]
def filter_strings_by_length(lst):
    return [item for item in lst if len(item) > 2]
dts=[','.join(filter_strings_by_length(d.words)) for d in documents]
data_prep.data['all_clear']=dts


In [ ]:
data_prep.data['all_clear']

In [ ]:
data_prep.data.to_csv('../data/cleared_columns.csv')